# 6. Ejemplo de inferencia en twitter para producción

![ping-data.gif](../assets/ping-data.gif)
    <sub>Data que genera un ping de una botnet a traves de todo el mundo.</sub>

# Moving Our Twitter Inference Towards Production

![](https://assets.bwbx.io/images/users/iqjWHBFdfxIU/inIpLScj8Lco/v1/-1x-1.jpg)

# The Question:

### What is the probability that a tweet originating from within Colombia contains at least 1 occurence of the word "yo" with any given composition of accents, and capital and lowercase letters?

# Gather data

In [3]:
import os
!pip install tweepy
import tweepy
from tweepy import Stream
print("Tweepy version {}".format(tweepy.__version__))

Tweepy version 3.5.0


### Load credentials

In [4]:
CONSUMER_KEY = "5Fk4fZDkTGuEGLvzQ3GBe3UEE"
CONSUMER_SECRET = "PudttRS8xOWzb6UqSuQKkAuQMVxG13qcdF9yOdd9NAVUkrXKta"
ACCESS_TOKEN = "521880142-52Qf9raiVKFVAmUy7kv89kNCaD27aD8giA1Ungsc"
ACCESS_TOKEN_SECRET = "VakaTTkNIWl6XYoRimPwRjCmRkuuGz0UWKA3pdgcddKfG"

In [5]:
COLOMBIA_GEO_LOCATION_BOUNDING_BOX = [-78.31, 0.44, -70.71, 11.39]

In [6]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

### Connect to the Twitter API

#### Define some cleaning functions

In [9]:
!pip install unidecode
from unidecode import unidecode


def make_lowercase(tweet):
    return tweet.lower()


def remove_diacritics(tweet):
    return unidecode(tweet)


def remove_non_alpha_characters(tweet):
    return ''.join(character for character in tweet if character.isalpha() or character == ' ')

#### Create a stream listener

In [11]:
DATABASE_NAME = 'twitter_inference'
TABLE_NAME = 'tweets'
USER = 'postgres'
HOST = 'localhost'
PASSWORD = 'marckzucaritas1'

In [13]:
!pip install psycopg2
import psycopg2
from tweepy import StreamListener


class PersistedStreamListener(StreamListener):
    
    def __init__(self):
        self._database_connection = psycopg2.connect(dbname=DATABASE_NAME, user=USER, host=HOST, password=PASSWORD)
        super().__init__()
    
    def on_status(self, status):
        cleaned_status_text = self._clean_status_text(status.text)
        self._insert_status(id_str=status.id_str, text=cleaned_status_text)
        
    def _clean_status_text(self, status_text):
        cleaned_status_text = status_text
        for cleaning_function in self._cleaning_functions:
            cleaned_status_text = cleaning_function(cleaned_status_text)
        return cleaned_status_text
    
    def _insert_status(self, id_str, text):
        cursor = self._database_connection.cursor()
        insert_statement = """INSERT INTO {table_name} VALUES ('{id_str}', '{text}')""".format(
                table_name=TABLE_NAME, id_str=id_str, text=text)

        cursor.execute(insert_statement)
        self._database_connection.commit()
        
        cursor.close()
        
    @property
    def _cleaning_functions(self):
        return [make_lowercase, remove_diacritics, remove_non_alpha_characters]

In [15]:
streaming_api = Stream(auth=auth, listener=PersistedStreamListener())

### Beta distributions

![](http://i.imgur.com/mj059cS.png)

# Creando nuestro modelo matematico

Vamos a construir un modelo matemático para responder nuestra pregunta original.

Lo primero que es importante explicar es una distribución beta, una distribución beta es una distribución de probabilidad canónica usada para mostrar incertidumbre alrededor de cada posible probabilidad, una probabilidad por supuesto siendo un numero entre 0 y 1, siendo la verdadera probabilidad del proceso.

Entonces entremos a esto un poco más, de nuevo la pregunta es:

¿Cuál es la probabilidad de que un tweet dado en Colombia contenga la palabra “yo”?

Entonces la respuesta va a ser un número entre 0 y 1 por definición de lo que es una probabilidad
y esta distribución de probabilidad es una distribución beta es efectivamente una tabla de consulta
para los datos dados que hemos observado y por supuesto limpiada y persistida, etc.

Una distribución beta es una función que toma dos parámetros, el primero es alpha y el segundo beta, entonces para interpretar una distribución beta vemos la altura de la curva y la altura de la curva es proporcional a la probabilidad de dibujar el valor por debajo de él en el eje x.


### Stream tweets

In [16]:
streaming_api.filter(locations=COLOMBIA_GEO_LOCATION_BOUNDING_BOX, async=True)

In [18]:
import numpy as np
from scipy.stats import beta as beta_distribution


X_VALUES = np.linspace(0, 1, 1002)[1:-1]
DATABASE_CONNECTION = psycopg2.connect(dbname=DATABASE_NAME, user=USER, host=HOST, password=PASSWORD)
KEYWORD = 'yo'


def fetch_tweets(database_connection=DATABASE_CONNECTION):
    cursor = database_connection.cursor()
    select_statement = """SELECT text FROM {table}""".format(table=TABLE_NAME)
    cursor.execute(select_statement)
    result = cursor.fetchall()
    
    return [tweet[0] for tweet in result]


def compute_alpha_and_beta(tweets, keyword=KEYWORD):
    number_of_occurences = sum(keyword in tweet for tweet in tweets)
    alpha = 1 + number_of_occurences
    beta = 1 + (len(tweets) - number_of_occurences)
    
    return alpha, beta


def compute_pdf_y_values(alpha, beta, x_values=X_VALUES):
    return beta_distribution(alpha, beta).pdf(x_values)

# Automatización

Lo que no estamos haciendo es ejecutando el código de forma manual. Ahora pasaremos a automatizar el proceso, lo cual es sinónimo de llevar nuestras cosas a producción.

Crearemos un panel de control, repito, que se refresque solo. Para ello usaremos bokeh

Para usar Bokeh lo primero que tenemos que hacer es iniciar un servidor Bokeh. Para hacer eso vamos a ir a nuestra terminal.

bokeh serve

Lo segundo que vamos a hacer aquí es traer nuestros tweets y luego definir alfa y beta. Recuerden que alfa y beta se definen primero, calculando primero cuántos de estos tweets realmente contienen la palabra clave “yo”, la cual nos interesa. Y continuación alfa se define como uno más el número de ocurrencias positivas. Y beta se define como uno más el número de ocurrencias negativas.

Después generamos código para esta figura en sí. De nuevo, todo esto está en la documentación de Bokeh. Creamos este objeto figure. Le damos un título, una etiqueta para el eje x, una etiqueta para el eje y, dimensiones en nuestro cuaderno interactivo.

Y finalmente, y esto es digamos que la sustancia aquí, definimos esta función update. Llamamos esta función update cada 50 milisegundos y lo que hace cada 50 milisegundos es:

 * Trae tweets
 * Redefine alfa y beta.
 * Recalcula
 * Agrega estos nuevos datos a nuestra visualización.

Pero por supuesto la idea aquí es automatización, esa es realmente la palabra clave. Que cada vez que queramos ver, ya sabes, cómo se ve esta curva después de observar muchos más datos
sólo hay un lugar en el cual buscar. Y lo que no voy a hacer es volver a ejecutar manualmente este análisis. Sólo estoy mirando mi modelo llevado a producción, obteniendo estos resultados y realmente pasando a lo siguiente.


In [11]:
from bokeh.client import push_session
from bokeh.models import FixedTicker
from bokeh.plotting import figure, curdoc, reset_output

# reset output
reset_output()

# initialize alpha, beta
tweets = fetch_tweets()
alpha, beta = compute_alpha_and_beta(tweets=tweets)
pdf_y_values = compute_pdf_y_values(alpha, beta)

# create bokeh figure
bokeh_figure = figure(
    title='PDF of True Probability of a Tweet Containing Keyword',
    x_axis_label='true_probability',
    y_axis_label='probability_density',
    width=1000,
    height=600
)
bokeh_figure.xaxis[0].ticker=FixedTicker(ticks=list(np.linspace(0, 1, 21)))
bokeh_line = bokeh_figure.line(X_VALUES, pdf_y_values, color="navy", line_width=4)

# open a session to keep our local document in sync with server
session = push_session(curdoc())

def update():
    tweets = fetch_tweets()
    alpha, beta = compute_alpha_and_beta(tweets=tweets)
    pdf_y_values = compute_pdf_y_values(alpha, beta)
    bokeh_line.data_source.data.update(y=pdf_y_values)

curdoc().add_periodic_callback(update, 50)

session.show(bokeh_figure)

session.loop_until_closed()